Inga Ulusoy, Computational modelling in python, SoSe2020 

# The quantum harmonic oscillator I and anharmonicity

Reference:
https://doi.org/10.1021/acs.jchemed.7b00003

The quantum harmonic oscillator is another standard problem in Quantum Mechanics. However, contrary to the particle in a box, it is widely and very successfully used as a model to describe molecular vibrations. For example, all standard quantum chemistry programs use the harmonic oscillator approximation in their frequency analysis. The advantage of this approximation is that the knowledge of the second derivatives at the optimized structure of a molecule is sufficient to calculate the IR frequencies with which that molecule vibrates. These frequencies can then be compared to spectral data and help identify the type of bonds and structural motif (isomer) of a molecule.

The potential energy term is approximated as a harmonic potential, where in the classical analogue the force acting on the particle is directly proportional to the displacement from equilibrium
\begin{align}
V = \frac{1}{2} k x^2
\end{align}
with the force constant $k=\mu \omega^2$ for a reduced mass $\mu$ and vibrational frequency $\omega$. The complete Hamiltonian thus reads
\begin{align}
\hat{H} = \hat{T} + V(x) = -\frac{\hbar^2}{2\mu} \frac{\partial^2}{\partial x^2} + \frac{1}{2} k x^2
\end{align}
As an example, we will look at HCl with the reduced mass 
\begin{align}
\mu =\frac{m_H m_{Cl}}{m_H+m_{Cl}}
\end{align}
First, we need to define and calculate some basic parameters and functions. We will use the physical constants from scipy; for a comprehensive list see https://docs.scipy.org/doc/scipy/reference/constants.html.

It is more convenient and numerically accurate to use atomic units. Furthermore, we will use mass-weighted coordinates $R=x/\sqrt{\mu}$, leading to the more compact Hamiltonian
\begin{align}
\hat{H} = \hat{T} + V(R) = -\frac{\hbar^2}{2} \frac{\partial^2}{\partial R^2} + \frac{1}{2} \omega^2 R^2
\end{align}

For now, we will look at the eigenstates of the HO and solve the time-independent Schrödinger equation (TISE).

In [ ]:
from numpy import *
from scipy import linalg
from scipy.constants import eV, c, h, hbar, m_e
from scipy.constants import physical_constants
import matplotlib.pyplot as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
amu = physical_constants['atomic mass constant'][0]
bohr = physical_constants['Bohr radius'][0]
cm2Eh = 4.556335E-6
JtoEh = 4.359744E-18

In [ ]:
#parameters of the HO: no of grid points for x
nsteps=500
xmin = 0
xmax = 5
#vibrational frequency in cm^-1
w = 2990
#interatomic distance in a.u.
R0 = 2.9
omega = w*cm2Eh

def get_mu(m1,m2):
    mu = m1*m2/(m1+m2)
    return mu

def get_k(mu,omega):
    k = mu*(2*pi*omega*100*c)**2
    #print(omega*100*c*10**-12,'THz')
    return k

m1 = 1*amu
m2 = 35*amu
mu = get_mu(m1,m2)
#convert this to atomic units by dividing througe m_e
muau=mu/m_e
k = get_k(mu,w)
print('Reduced mass is {:3.2e} kg ({:4.2f} au) and the force constant {:4.2f} N/m.'.format(mu,muau,k))

#set up the nuclear grid
#step size h for the numerical derivative
xgrid,h=linspace(xmin,xmax,nsteps,retstep=True)

def potential(x,omega):
    pot=(0.5*omega**2*x**2)
    return pot

# create the Hamiltonian
Hamiltonian=zeros((nsteps,nsteps))
[i,j] = indices(Hamiltonian.shape)
Laplacian=(-2.0*diag(ones(nsteps))+diag(ones(nsteps-1),1)+diag(ones(nsteps-1),-1))/(float)(h**2)
#convert the internuclear distance to relative distance from R0
xgridr = (xgrid-R0)*sqrt(muau)
V=potential(xgridr,omega)
Hamiltonian[i==j]=V
Hamiltonian+=(-1.0/(2.0*muau))*Laplacian

energies, wavef = linalg.eigh(Hamiltonian)
print('Zero-point energy: {:3.2f} cm-1'.format(energies[0]/cm2Eh))
print('Fundamental vibration: {:3.2f} cm-1'.format((energies[1]-energies[0])/cm2Eh))

In [ ]:
mf=18
fig, ax = plt.subplots(figsize=(8,8))
ax.plot(xgrid,wavef[:,0]*3000+energies[0]/cm2Eh)
ax.plot(xgrid,wavef[:,1]*3000+energies[1]/cm2Eh)
ax.plot(xgrid,wavef[:,2]*3000+energies[2]/cm2Eh)

ax.plot(xgrid,V/cm2Eh)
ax.set_xlim(left=2.25,right=3.6)
ax.set_ylim(bottom=0,top=10000)
ax.set_xlabel("x (a$_0$)",fontsize = mf)
ax.set_ylabel("Energy (cm$^{-1}$)",fontsize = mf)
ax.xaxis.set_tick_params(labelsize=mf)
ax.yaxis.set_tick_params(labelsize=mf)

plt.show()

# Now let's compare this to the AHO

In the anharmonic oscillator (AHO), the force is no longer directly proportional to the displacement, and the potential is asymmetric - dissociation is allowed in the AHO. The anharmonic potential for a diatomic is often approximated as a Morse potential
\begin{align}
    V(R) = D_{e} \left[ 1-\exp\left(-\beta(R-R_0)\right)\right]^2
\end{align}

Here, $D_e$ is the potential depth ($7.41\cdot10^{-19}$ J for HCl) and $\beta=\sqrt{k/2D_e}$.

In [ ]:
De = 7.41E-19
beta = sqrt(k/(2*De))
#betaau = beta/bohr
#Deau = De/JtoEh
#Decm = Deau/cm2Eh
#print(Deau,Decm,beta/1E10,betaau)
def Morse(xm,De,beta):
    pot=De*(1.0-exp(-beta*xm))**2
    #this way minimum is at zero
    #pot= De*(exp(-2.0*beta*xm)-2.0*exp(-beta*xm))
    return pot

In [ ]:
xd = (xgrid - R0)*bohr
VM=Morse(xd,De,beta)
#VM=potential(xd,omega)
#xgridr = (xgrid-R0)*sqrt(muau)
#VM=potential(xgridr,omega)
Hamiltonian=zeros((nsteps,nsteps))
[i,j] = indices(Hamiltonian.shape)
Laplacian=(-2.0*diag(ones(nsteps))+diag(ones(nsteps-1),1)+diag(ones(nsteps-1),-1))/(float)(h**2)
Hamiltonian[i==j]=VM/JtoEh
Hamiltonian+=(-1.0/(2.0*muau))*Laplacian

energiesM, wavefM = linalg.eigh(Hamiltonian)
print('Zero-point energy: {:3.2f} cm-1'.format(energiesM[0]/cm2Eh))
print('Fundamental vibration: {:3.2f} cm-1'.format((energiesM[1]-energiesM[0])/cm2Eh))

In [ ]:
#overview of the potential
xmin = 0
xmax = 15
tempx = linspace(xmin,xmax,nsteps)
tempR = (tempx - R0)*bohr
tempV=Morse(tempR,De,beta)

fig, ax = plt.subplots(figsize=(8,5))
ax.plot(tempx,tempV/(JtoEh*cm2Eh))
ax.set_xlabel("x (a$_0$)",fontsize = mf)
ax.set_ylabel("Energy (cm$^{-1}$)",fontsize = mf)
ax.set_ylim(bottom=0,top=100000)
ax.xaxis.set_tick_params(labelsize=mf)
ax.yaxis.set_tick_params(labelsize=mf)
ax.hlines(De/(JtoEh*cm2Eh),tempx[0],tempx[-1],linestyle=':')
hl=5000
ax.arrow(R0,0,0,De/(JtoEh*cm2Eh)-hl,head_width=0.2, head_length=hl, fc='k', ec='k')
plt.show()
#Shown is De, D0 would be De - EZPVE

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
#ax.plot(xgrid,wavefM[:,0])
ax.plot(xgrid,wavefM[:,0]*3000+energiesM[0]/cm2Eh)
ax.plot(xgrid,wavefM[:,1]*3000+energiesM[1]/cm2Eh)
ax.plot(xgrid,wavefM[:,2]*3000+energiesM[2]/cm2Eh)

ax.plot(xgrid,VM/(JtoEh*cm2Eh))
ax.plot(xgrid,V/cm2Eh)

#ax.plot(xd,VM/cm2Eh)
#ax.set_xlim(left=2,right=4)
ax.set_xlim(left=2.25,right=4.5)
ax.set_ylim(bottom=0,top=20000)
ax.set_xlabel("x (a$_0$)",fontsize = mf)
ax.set_ylabel("Energy (cm$^{-1}$)",fontsize = mf)
ax.xaxis.set_tick_params(labelsize=mf)
ax.yaxis.set_tick_params(labelsize=mf)

#ax.set_ylim(bottom=0,top=1)
plt.show()

# Task 1

Compare the eigenenergies of the HO and AHO with increasing quantum number. What trend do you see? Provide your answer on moodle.

# Task 2

Calculate the expectation value for x and p for the first four HO and AHO wavefunctions. Generate a plot and upload to moodle.

# Optional task

Add the quantum HO and AHO case to your program.